In [1]:
from findata_tools import *

In [2]:
import numpy as np
import pandas as pd

In [3]:
%pwd

'/Users/yg/code/github/DS-GA-1013/hw/hw2/findata'

In [4]:
names, days = load_data('stockprices.csv')
(n_days, n_stocks) = days.shape
print(days.shape)
n_returns = n_days - 1
returns = days[1:] - days[:-1]
print(returns.shape, returns)

(433, 18)
(432, 18) [[ -2.550804  -3.200012   0.239204 ...   0.26794   -0.239998  -0.32    ]
 [ -1.942085  -1.139954  -0.956821 ...  -1.63744    2.079994  -0.55    ]
 [ -4.106422 -24.710022  -1.798817 ...  -2.828319   4.040001  -0.21    ]
 ...
 [ -1.210007 -12.599976  -0.149994 ...   0.369995  -0.159999  -0.03    ]
 [  0.059998  -4.330017   0.279998 ...   0.139999  -0.090001  -0.06    ]
 [ -2.659989   3.350037  -0.5      ...   0.080002  -0.039997   0.13    ]]


/Users/yg/code/github/DS-GA-1013/hw/hw2/findata/findata_tools.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = df.as_matrix()


In [5]:
centered_returns = returns - np.mean(returns, axis=0)
cov = np.cov(centered_returns,rowvar=False)
U, S, principal_directions = np.linalg.svd(cov)
max_coeff_stocks_index = [np.argmax(np.abs(principal_directions[k])) for k in range(2)]
max_coeff_stocks = [names[k] for k in max_coeff_stocks_index]

print(max_coeff_stocks)
pretty_print(np.sum(np.abs(returns),axis = 0),names)

['amzn', 'goog']
       aapl         amzn        msft         goog         xom         apc  \
  454.14342  3423.779352  208.844523  2799.940424  258.501684  388.899131   

         cvx           c          gs       jpm         aet         jnj  \
  381.849154  247.140956  901.311051  277.9143  505.556593  275.823333   

         dgx         spy        xlf         sso         sds    uso  
  257.409551  424.583189  64.368439  277.841774  276.939997  77.16  


In [6]:
print(names)

['aapl', 'amzn', 'msft', 'goog', 'xom', 'apc', 'cvx', 'c', 'gs', 'jpm', 'aet', 'jnj', 'dgx', 'spy', 'xlf', 'sso', 'sds', 'uso']


In [7]:
df = pd.DataFrame(list(np.sum(np.abs(returns),axis = 0)))
df = df.rename(columns={0: "return"})
df["stock"] = names
df.sort_values("return", inplace=True)
df

,return,stock
14,64.368439,xlf
17,77.160000,uso
2,208.844523,msft
7,247.140956,c
12,257.409551,dgx
4,258.501684,xom
11,275.823333,jnj
16,276.939997,sds
15,277.841774,sso
9,277.914300,jpm


In [8]:
print("{}, {}, {}, {}"
      .format(3423.779352/901.311051, 3423.779352/64.368439 , 2799.940424 / 901.311051, 2799.940424 /64.368439 ))

3.798665675075585, 53.19034304995341, 3.106519576003734, 43.49865349383415


# 3 - B Question

In [9]:
# for k in range(2):
#     print("PD")
#     pretty_print(v2[:, k],names)

In [10]:
U, S, principal_directions_stand = np.linalg.svd(np.corrcoef(centered_returns,rowvar=False))
for k in range(2):
    print("PD{}".format(k))
    pretty_print(principal_directions_stand[k],names)

PD0
      aapl      amzn      msft      goog       xom       apc       cvx  \
 -0.184648 -0.168095 -0.227645 -0.197887 -0.208861 -0.194808 -0.223865   

         c        gs      jpm       aet       jnj       dgx       spy  \
 -0.280147 -0.260443 -0.27294 -0.188555 -0.150783 -0.196612 -0.336632   

       xlf       sso       sds       uso  
 -0.198364 -0.334826  0.327259 -0.159213  
PD1
      aapl    amzn      msft      goog       xom       apc       cvx  \
  0.285821  0.3302  0.302005  0.389346 -0.293444 -0.329116 -0.308272   

         c        gs       jpm       aet       jnj       dgx       spy  \
 -0.133288 -0.137664 -0.155599  0.071286  0.108239  0.165412  0.084872   

       xlf       sso       sds       uso  
 -0.124933  0.091568 -0.056002 -0.370943  


In [11]:
def extract_stock_price(index_day, prices, index_stock, names):
    stock_name = names[index_stock]
    daily_price = prices[index_day,:]
    stock_price = daily_price[index_stock]
    return stock_price, [stock_name]

def extract_stock_weights(index_stock_end, principal_direction, principal_direction_index, 
                          index_day, prices, names, index_stock_start=None):
    if index_stock_start:
        weigth_stocks = principal_direction[principal_direction_index, index_stock_start:index_stock_end]
        stock_names = names[index_stock_start:index_stock_end]
        daily_prices = prices[index_day,:]
        stock_prices = daily_prices[index_stock_start:index_stock_end]
    else:
        weigth_stocks = principal_direction[principal_direction_index, :index_stock_end]
        stock_names = names[:index_stock_end]
        daily_prices = prices[index_day,:]
        stock_prices = daily_prices[:index_stock_end]
    return daily_prices, weigth_stocks, stock_prices, stock_names

def compute_weighted_average(weigth_stocks, price_stocks):
    return np.sum(np.multiply(weigth_stocks, price_stocks), axis=0)/ np.sum(weigth_stocks, axis=0)

# SPY

State Street's SPDR S&P 500 ETF.  A security that roughly tracks the S&P 500, a weighted average of the stock prices of 500 top US companies.

In [12]:
index_day = 0
index_stock = 13
index_stock_end = index_stock
principal_direction_index = 0
daily_prices, weigth_stocks, stock_prices, stock_names = extract_stock_weights(index_stock_end, principal_directions_stand, principal_direction_index,
                                                                                index_day, days, names)
print(80*"-")
print("Weights for the first 13 stock prices using PD{}".format(principal_direction_index))
pretty_print(weigth_stocks, stock_names)
print(80*"-")
print("Absolute values of weights for the first 13 stock prices using PD{}".format(principal_direction_index))
pretty_print(np.abs(weigth_stocks), stock_names)
print(80*"-")
print("Day{} prices".format(index_day))
pretty_print(stock_prices, stock_names)
print(80*"-")
print("Day{}, computed SPY weighted average prices:{}".format(index_day, compute_weighted_average(np.abs(weigth_stocks), stock_prices)))
print(80*"-")
print("Day{} SPY price".format(index_day))
spy_weight = principal_directions_stand[principal_direction_index, index_stock]
print("SPY weight:{}".format(spy_weight))
spy_return, spy_name = extract_stock_price(index_day, days, index_stock, names)
pretty_print(spy_return, spy_name)

--------------------------------------------------------------------------------
Weights for the first 13 stock prices using PD0
      aapl      amzn      msft      goog       xom       apc       cvx  \
 -0.184648 -0.168095 -0.227645 -0.197887 -0.208861 -0.194808 -0.223865   

         c        gs      jpm       aet       jnj       dgx  
 -0.280147 -0.260443 -0.27294 -0.188555 -0.150783 -0.196612  
--------------------------------------------------------------------------------
Absolute values of weights for the first 13 stock prices using PD0
      aapl      amzn      msft      goog       xom       apc       cvx  \
  0.184648  0.168095  0.227645  0.197887  0.208861  0.194808  0.223865   

         c        gs      jpm       aet       jnj       dgx  
  0.280147  0.260443  0.27294  0.188555  0.150783  0.196612  
--------------------------------------------------------------------------------
Day0 prices
        aapl       amzn       msft        goog        xom        apc  \
  101.790649

# XLF: State Street's SPDR Financials ETF.  A security that tracks a weighted average of top US financial companies.

In [13]:
for k in range(2):
    print("{} PD".format(k+1))
    pretty_print(principal_directions_stand[k],names)

1 PD
      aapl      amzn      msft      goog       xom       apc       cvx  \
 -0.184648 -0.168095 -0.227645 -0.197887 -0.208861 -0.194808 -0.223865   

         c        gs      jpm       aet       jnj       dgx       spy  \
 -0.280147 -0.260443 -0.27294 -0.188555 -0.150783 -0.196612 -0.336632   

       xlf       sso       sds       uso  
 -0.198364 -0.334826  0.327259 -0.159213  
2 PD
      aapl    amzn      msft      goog       xom       apc       cvx  \
  0.285821  0.3302  0.302005  0.389346 -0.293444 -0.329116 -0.308272   

         c        gs       jpm       aet       jnj       dgx       spy  \
 -0.133288 -0.137664 -0.155599  0.071286  0.108239  0.165412  0.084872   

       xlf       sso       sds       uso  
 -0.124933  0.091568 -0.056002 -0.370943  


In [14]:
index_day = 100
index_stock = 14
index_stocks_start = 7
index_stocks_end = 10
principal_direction_index = 0
daily_prices, weigth_stocks, stock_prices, stock_names = extract_stock_weights(index_stocks_end, principal_directions_stand, principal_direction_index,
                                                                                index_day, days, names,index_stocks_start)
print(80*"-")
print("Weights for the 3 financial stock prices using PD{}".format(principal_direction_index))
pretty_print(weigth_stocks, stock_names)
print(80*"-")
print("Absolute values of weights for the 3 financial stock prices using PD{}".format(principal_direction_index))
pretty_print(np.abs(weigth_stocks), stock_names)
print(80*"-")
print("Day{} prices".format(index_day))
pretty_print(stock_prices, stock_names)
print(80*"-")
print("Day{}, computed XLF weighted average prices:{}".format(index_day, compute_weighted_average(weigth_stocks, stock_prices)))
print(80*"-")
print("Day{} XLF price".format(index_day))
xlf_weight = principal_directions_stand[principal_direction_index, index_stock]
print("XLF weight:{}".format(xlf_weight))
stock_price, stock_name = extract_stock_price(index_day, days, index_stock, names)
print("Day{} XLF price".format(index_day))
pretty_print(stock_price, stock_name)

--------------------------------------------------------------------------------
Weights for the 3 financial stock prices using PD0
         c        gs      jpm
 -0.280147 -0.260443 -0.27294
--------------------------------------------------------------------------------
Absolute values of weights for the 3 financial stock prices using PD0
         c        gs      jpm
  0.280147  0.260443  0.27294
--------------------------------------------------------------------------------
Day100 prices
          c          gs        jpm
  45.329781  155.994034  62.998978
--------------------------------------------------------------------------------
Day100, computed XLF weighted average prices:86.68575537909776
--------------------------------------------------------------------------------
Day100 XLF price
XLF weight:-0.1983641702424083
Day100 XLF price
        xlf
  14.347868


# SSO ProShares levered ETF that roughly corresponds to twice the daily performance of the S&P 500.

In [15]:
index_day=200
index_stock = -1
daily_return = returns[index_day,:]
stock_returns = daily_return[:]
pretty_print(stock_returns, names)

      aapl      amzn      msft     goog       xom       apc       cvx  \
 -0.343941  0.039978 -0.126941  6.23999  0.385491  0.956391  0.461364   

         c        gs       jpm       aet       jnj      dgx       spy  \
  0.483455  1.856705  0.639229 -0.366638 -0.798614 -0.71032  0.558686   

       xlf      sso       sds   uso  
  0.167292  0.33892 -0.240006  0.21  


In [16]:
index_day=201
daily_return = returns[index_day,:]
stock_returns = daily_return[:]
pretty_print(stock_returns, names)

     aapl      amzn      msft      goog       xom       apc       cvx  \
 -0.05896 -7.369995 -0.273407 -4.530029  0.038551 -0.169361 -0.384475   

         c   gs       jpm       aet       jnj       dgx      spy  xlf  \
  0.098664  0.0 -0.088517 -0.128807  0.272697  1.568627 -0.39206  0.0   

       sso       sds   uso  
 -0.199363  0.200005 -0.23  


# SSO ProShares levered ETF that roughly corresponds to twice the daily performance of the S&P 500.

In [21]:
print("SSO roughly twice daily performance of SPY") 
print("First PD, SSO:{}, SPY:{}".format(principal_directions_stand[0, 15], principal_directions_stand[0, 13]))
print("Second PD, SSO:{}, SPY:{}".format(principal_directions_stand[1, 15], principal_directions_stand[1, 13]))

SSO roughly twice daily performance of SPY
First PD, SSO:-0.3348264259722335, SPY:-0.336631824619049
Second PD, SSO:0.0915676628910389, SPY:0.08487248667261033


# SDS ProShares inverse levered ETF that roughly corresponds to 
    twice the negative daily performance of the S&P 500.  That is,
    when the S&P 500 goes up by a dollar, this roughly goes down by 2.

In [24]:
for k in range(2):
    print("{} PD".format(k+1))
    pretty_print(principal_directions_stand[k],names)

1 PD
      aapl      amzn      msft      goog       xom       apc       cvx  \
 -0.184648 -0.168095 -0.227645 -0.197887 -0.208861 -0.194808 -0.223865   

         c        gs      jpm       aet       jnj       dgx       spy  \
 -0.280147 -0.260443 -0.27294 -0.188555 -0.150783 -0.196612 -0.336632   

       xlf       sso       sds       uso  
 -0.198364 -0.334826  0.327259 -0.159213  
2 PD
      aapl    amzn      msft      goog       xom       apc       cvx  \
  0.285821  0.3302  0.302005  0.389346 -0.293444 -0.329116 -0.308272   

         c        gs       jpm       aet       jnj       dgx       spy  \
 -0.133288 -0.137664 -0.155599  0.071286  0.108239  0.165412  0.084872   

       xlf       sso       sds       uso  
 -0.124933  0.091568 -0.056002 -0.370943  


In [25]:
print("SDS roughly twice daily the negative daily performance of SPY") 
print("First PD, SSO:{}, SPY:{}".format(principal_directions_stand[0, 16], principal_directions_stand[0, 13]))
print("Second PD, SSO:{}, SPY:{}".format(principal_directions_stand[1, 16], principal_directions_stand[1, 13]))

SDS roughly twice daily the negative daily performance of SPY
First PD, SSO:0.32725941343597814, SPY:-0.336631824619049
Second PD, SSO:-0.05600186500392546, SPY:0.08487248667261033


# USO Exchange traded product that tracks the price of oil in the US.

In [26]:
index_day=200
index_stock = -1
daily_return = returns[index_day,:]
stock_returns = daily_return[:]
pretty_print(stock_returns, names)

      aapl      amzn      msft     goog       xom       apc       cvx  \
 -0.343941  0.039978 -0.126941  6.23999  0.385491  0.956391  0.461364   

         c        gs       jpm       aet       jnj      dgx       spy  \
  0.483455  1.856705  0.639229 -0.366638 -0.798614 -0.71032  0.558686   

       xlf      sso       sds   uso  
  0.167292  0.33892 -0.240006  0.21  


In [ ]:
for k in range(2):
    print("{} PD".format(k+1))
    pretty_print(principal_directions_stand[k],names)

In [27]:
print("Rough correlation of USO that tracks price of US oil with the mean of the US oil companies among 18 stocks") 
print("First PD, USO:{}, rough average:{}".format(principal_directions_stand[0, -1], principal_directions_stand[0, 4:7].mean()))
print("Second PD, USO:{}, rough average:{}".format(principal_directions_stand[1, -1], principal_directions_stand[1, 4:7].mean()))

Rough correlation of USO that tracks price of US oil with the mean of the US oil companies among 18 stocks
First PD, USO:-0.15921267314694082, rough average:-0.20917793731888332
Second PD, USO:-0.3709433376773426, rough average:-0.31027749841006097


# Q3 - C

In [30]:
shares = [100 for _ in range(n_stocks)]
double_shares_stocks = ["aapl","amzn","msft","goog"]
shares = np.array([2 * shares[i] if names[i] in double_shares_stocks else shares[i] for i in range(n_stocks)]).reshape(-1,1)
np.sqrt(np.dot(shares.T, np.matmul(cov, shares))).item()

4309.949525975801

# Q4 - d

In [31]:
daily_portfolio_returns = np.dot(returns, shares) 
p = sum([daily_portfolio_returns[i] <= -1000 for i in range(n_returns)])/n_returns
print(p.item())

0.3425925925925926


In [ ]:
daily_portfolio_returns.shape

In [ ]:
n_returns

In [ ]:
np.std(daily_portfolio_returns)